### Notebook for adult stem cells batch correction with `scVI` 

- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Created date:** 15th April 2024
- **Last modified date:** 15th April 2024

##### Import packages

In [ ]:
import scvi
import torch
import anndata
import warnings
import numpy as np
import scanpy as sc
import pandas as pd
import plotnine as p
from pywaffle import Waffle
import matplotlib.pyplot as plt
from scib_metrics.benchmark import Benchmarker

##### Setup working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

In [ ]:
warnings.simplefilter(action = 'ignore')
scvi.settings.seed = 1712
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"}
%config InlineBackend.figure_format = 'retina'
torch.set_float32_matmul_precision('medium')

In [ ]:
arches_params = dict(
    use_layer_norm = "both",
    use_batch_norm = "none",
    encode_covariates = True,
    dropout_rate = 0.2,
    n_layers = 3,
)

In [ ]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

### Read in data

+ Reference data (fetal stem cells)

In [ ]:
input = '/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/Adult_stem_cells/Adult_stem_cells_2K_round4.h5ad'
adata = sc.read_h5ad(input)

In [ ]:
X_is_raw(adata)

- Save raw counts to anndata

In [ ]:
adata_raw = adata.copy()

### Calculate HVGs

In [ ]:
adata.layers['counts'] = adata.X.copy()

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 7000,
    layer = "counts",
    batch_key = "Donor_ID",
    subset = True,
    span = 1
)
adata

### Data integration with `scVI`

- Create a model

In [ ]:
scvi.model.SCVI.setup_anndata(adata, 
                              categorical_covariate_keys=['Donor_ID', 'Library_Preparation_Protocol', 'Study_name'],
                              labels_key = "C_scANVI", 
                              layer = 'counts')

In [ ]:
scvi_model = scvi.model.SCVI(adata,
                            n_latent = 256, 
                            n_layers = 3, 
                            dispersion = 'gene-batch', 
                            gene_likelihood = 'nb', n_hidden = 1000)

- Train a model

In [ ]:
scvi_model.train(33, 
                 check_val_every_n_epoch = 1, 
                 enable_progress_bar = True, 
                 accelerator = "gpu",
                 devices = [0])

- Save latent representations

In [ ]:
adata.obsm["X_scVI"] = scvi_model.get_latent_representation(adata)

#### Evaluate model performance using the [_Svensson_](https://www.nxn.se/valent/2023/8/10/training-scvi-posterior-predictive-distributions-over-epochs) method

In [ ]:
history_df = (
    scvi_model.history['elbo_train'].astype(float)
    .join(scvi_model.history['elbo_validation'].astype(float))
    .reset_index()
    .melt(id_vars = ['epoch'])
)

p.options.figure_size = 12, 6

p_ = (
    p.ggplot(p.aes(x = 'epoch', y = 'value', color = 'variable'), history_df.query('epoch > 0'))
    + p.geom_line()
    + p.geom_point()
    + p.scale_color_manual({'elbo_train': 'black', 'elbo_validation': 'red'})
    + p.theme_minimal()
)

print(p_)

- Calculate UMAP

In [ ]:
sc.pp.neighbors(adata, use_rep = "X_scVI", n_neighbors = 50, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.3, spread = 1, random_state = 1712)

In [ ]:
sc.set_figure_params(dpi = 300, figsize=(10,7))
sc.pl.umap(adata, color = ['Study_name', 'C_scANVI', 'Library_Preparation_Protocol', 'Donor_ID'], ncols = 3, frameon = False, size = 5)

+ Export anndata object

In [ ]:
adata

In [ ]:
adata_export = anndata.AnnData(X = adata_raw.X, obs = adata.obs, var = adata_raw.var, uns = adata.uns, obsm = adata.obsm, obsp = adata.obsp)

In [ ]:
adata_export.write_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/Adult_stem_cells/Adult_stem_cells.h5ad')

+ Check the scIB metrics

In [ ]:
from rich import print
import scib
import scib.metrics

In [ ]:
from scib_metrics.benchmark import Benchmarker

In [ ]:
bm = Benchmarker(
    adata,
    batch_key='Donor_ID',
    label_key='C_scANVI',
    embedding_obsm_keys=['X_scVI', 'X_pca', 'X_umap'],
    n_jobs=-1
)

bm.benchmark()

In [ ]:
bm.plot_results_table(min_max_scale=False)

In [ ]:
bm = Benchmarker(
    adata,
    batch_key='Study_name',
    label_key='C_scANVI',
    embedding_obsm_keys=['X_scVI', 'X_pca', 'X_umap'],
    n_jobs=-1
)

bm.benchmark()

In [ ]:
bm.plot_results_table(min_max_scale=False)